# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [ ]:
# Write your code below.

%load_ext dotenv
%dotenv 

In [ ]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [ ]:
import os
from glob import glob

price_data = glob(os.path.join(os.getenv('PRICE_DATA'), "*/*/*.parquet"), recursive = True) # glob adds all files into a list
price_data

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
import dask.dataframe as dd

df = dd.read_parquet(price_data)


,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year
npartitions=2923,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,int64,object,object,int32
,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...


In [ ]:
# Create lags columns

df['Close_lag'] = df.groupby(['ticker'])['Close'].shift(1)
df['Adj_Close_lag'] = df.groupby(['ticker'])['Adj Close'].shift(1)

C:\Users\Andrea\AppData\Local\Temp\ipykernel_14152\1269309925.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  df['Close_lag'] = df.groupby(['ticker'])['Close'].shift(1)
C:\Users\Andrea\AppData\Local\Temp\ipykernel_14152\1269309925.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  df['Adj_Close_lag'] = df.groupby(['ticker'])['Adj Close'].shift(1)


,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag,Adj_Close_lag
npartitions=2923,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,int64,object,object,int32,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...


In [66]:
# Create Returns column: Close / Close_lag - 1

df['returns'] = df['Close']/df['Close_lag'] - 1
df


,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag,Adj_Close_lag,returns
npartitions=2923,,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,int64,object,object,int32,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...


In [67]:
# Create High-Low Range column: High - Low

df['hi_lo_range'] = df['High']-df['Low']
df

,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag,Adj_Close_lag,returns,hi_lo_range
npartitions=2923,,,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,int64,object,object,int32,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [ ]:
# Assign dataframe to dd_feat

dd_feat = df
dd_feat

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [ ]:
# Convert Dask dataframe to Pandas dataframe

pd_df = dd_feat.compute()

In [ ]:
# Perform Rolling Average calculation

pd_df.groupby('ticker').apply(
    lambda x: x.assign(Moving_Avg = x['Returns'].rolling(10).mean())
)

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Answer: We do not need to convert it to pandas to run the caluculation. Dask provides us a more efficient way to run the calculations.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.